In [5]:

# To model and solve a power generation optimization problem using Gurobi in Python, you'll need to formulate your 
# problem as a mathematical optimization model. Here's a simplified example using linear programming. In a real-world 
# scenario, you might need a more sophisticated model depending on the specifics of your system.

# Let's assume you have data for water flow, weather, temperature, sunshine, rainfall over the past year, and you 
# want to maximize power generation based on these factors.

# Essentialls, gurobipy is the Python interface for Gurobi Optimization, a powerful optimization solver. 
# Gurobi Optimization is a commercial optimization solver that provides state-of-the-art performance for a wide 
# range of mathematical programming problems, including linear programming (LP), quadratic programming (QP), 
# and mixed-integer programming (MIP).


In [5]:

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Sample data for the past year
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    'Water_Flow': [100, 120, 90, 80, 110, 130, 150, 140, 120, 100, 80, 90],
    'Temperature': [15, 18, 20, 25, 28, 30, 32, 28, 25, 20, 18, 16],
    'Sunshine': [6, 7, 8, 9, 10, 11, 12, 11, 10, 8, 7, 6],
    'Rainfall': [50, 40, 30, 20, 10, 5, 5, 10, 15, 25, 35, 45],
}

df = pd.DataFrame(data)
display(df)

# Baseline power generation considering average contributions from each source
# Let's assume some arbitrary but reasonable weights for hydro, solar, and wind
hydro_baseline = df['Water_Flow'] * 0.2
solar_baseline = df['Sunshine'] * 5
wind_baseline = (df['Temperature'] + df['Sunshine']) * 0.3

# Total baseline power generation
df['Baseline_Power_Gen'] = hydro_baseline + solar_baseline + wind_baseline

# Print baseline power generation
print("Baseline Power Generation:")
print(df[['Month', 'Baseline_Power_Gen']])


,Month,Water_Flow,Temperature,Sunshine,Rainfall
0,Jan,100,15,6,50
1,Feb,120,18,7,40
2,Mar,90,20,8,30
3,Apr,80,25,9,20
4,May,110,28,10,10
5,Jun,130,30,11,5
6,Jul,150,32,12,5
7,Aug,140,28,11,10
8,Sep,120,25,10,15
9,Oct,100,20,8,25


Baseline Power Generation:
   Month  Baseline_Power_Gen
0    Jan                56.3
1    Feb                66.5
2    Mar                66.4
3    Apr                71.2
4    May                83.4
5    Jun                93.3
6    Jul               103.2
7    Aug                94.7
8    Sep                84.5
9    Oct                68.4
10   Nov                58.5
11   Dec                54.6


In [6]:

# Create a Gurobi model
model = gp.Model("power_generation")

# Variables
months = df['Month'].tolist()
water_flow = df['Water_Flow'].tolist()
temperature = df['Temperature'].tolist()
sunshine = df['Sunshine'].tolist()
rainfall = df['Rainfall'].tolist()

# Decision variables for power generation from hydro, solar, and wind in each month
power_hydro = model.addVars(months, name="Hydro_Power", lb=0)
power_solar = model.addVars(months, name="Solar_Power", lb=0)
power_wind = model.addVars(months, name="Wind_Power", lb=0)


In [7]:

# Objective: Maximize total power generation over the year
model.setObjective(gp.quicksum(power_hydro[month] + power_solar[month] + power_wind[month] for month in months), sense=GRB.MAXIMIZE)

# Constraints: Power generation based on environmental factors
for month in months:
    idx = months.index(month)
    model.addConstr(power_hydro[month] <= 0.3 * water_flow[idx], f"Hydro_Constraint_{month}")
    model.addConstr(power_solar[month] <= 0.6 * sunshine[idx], f"Solar_Constraint_{month}")
    model.addConstr(power_wind[month] <= 0.2 * (temperature[idx] + sunshine[idx]), f"Wind_Constraint_{month}")

# Optimize the model
model.optimize()

# Print the optimal solution
print("Optimal Power Generation:")
optimal_power_gen = {
    month: {
        'Hydro': power_hydro[month].x,
        'Solar': power_solar[month].x,
        'Wind': power_wind[month].x,
        'Total': power_hydro[month].x + power_solar[month].x + power_wind[month].x
    }
    for month in months
}
for month, power in optimal_power_gen.items():
    print(f"{month}: Hydro: {power['Hydro']} MW, Solar: {power['Solar']} MW, Wind: {power['Wind']} MW, Total: {power['Total']} MW")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 36 rows, 36 columns and 36 nonzeros
Model fingerprint: 0x290b9e03
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+01]
Presolve removed 36 rows and 36 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3200000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.320000000e+02
Optimal Power Generation:
Jan: Hydro: 30.0 MW, Solar: 3.5999999999999996 MW, Wind: 4.2 MW, Total: 37.800000000000004 MW
Feb: Hydro: 36.0 MW, Solar: 4.2 MW, Wind: 5.0 MW, Total: 45.2 MW
M

In [8]:

# Calculate overall power generation
optimal_total_power = sum(power['Total'] for power in optimal_power_gen.values())
baseline_total_power = df['Baseline_Power_Gen'].sum()

# Calculate savings
savings_percentage = ((baseline_total_power - optimal_total_power) / baseline_total_power) * 100

print(f"\nOverall Savings: {savings_percentage:.2f}%")



Overall Savings: 40.95%


In [9]:

# Comparing baseline and optimized power generation
comparison_df = pd.DataFrame({
    'Month': months,
    'Baseline_Power_Gen': df['Baseline_Power_Gen'],
    'Optimal_Power_Gen': [power['Total'] for power in optimal_power_gen.values()]
})

print("\nComparison of Baseline and Optimal Power Generation:")
print(comparison_df)



Comparison of Baseline and Optimal Power Generation:
   Month  Baseline_Power_Gen  Optimal_Power_Gen
0    Jan                56.3               37.8
1    Feb                66.5               45.2
2    Mar                66.4               37.4
3    Apr                71.2               36.2
4    May                83.4               46.6
5    Jun                93.3               53.8
6    Jul               103.2               61.0
7    Aug                94.7               56.4
8    Sep                84.5               49.0
9    Oct                68.4               40.4
10   Nov                58.5               33.2
11   Dec                54.6               35.0


In [10]:

# This code sample sets up an optimization model using Gurobi to maximize power generation over the year while 
# considering constraints based on water flow, temperature, sunshine, and rainfall. The coefficients in 
# the constraints can be adjusted to reflect the impact of each factor on power generation. The final output 
# shows the arbitrary power generation vs. the optimal power generation, and the overall savings by optimizing
# the process.


# Baseline Generation: We calculate the baseline power generation for each month based on assumed contributions from hydro, solar, and wind sources.

# Decision Variables: Separate variables for hydro, solar, and wind power generation.

# Constraints: Power generation constraints based on environmental factors for each source.

# Optimization: The objective is to maximize the total power generation while adhering to the constraints.

# Comparison: We compare the baseline power generation with the optimized power generation to calculate savings and understand the improvements.


In [11]:

# Let's assume a more relaistic scenario.

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Sample data for the past year
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    'Water_Flow': [100, 120, 90, 80, 110, 130, 150, 140, 120, 100, 80, 90],
    'Temperature': [15, 18, 20, 25, 28, 30, 32, 28, 25, 20, 18, 16],
    'Sunshine': [6, 7, 8, 9, 10, 11, 12, 11, 10, 8, 7, 6],
    'Rainfall': [50, 40, 30, 20, 10, 5, 5, 10, 15, 25, 35, 45],
    'Hydro_Price': [30, 28, 25, 26, 27, 29, 31, 30, 29, 27, 28, 26],  # Prices for hydro power ($/MW)
    'Solar_Price': [40, 42, 43, 44, 45, 47, 48, 46, 44, 42, 41, 39],  # Prices for solar power ($/MW)
    'Wind_Price': [35, 34, 33, 36, 38, 37, 39, 38, 36, 34, 33, 32]   # Prices for wind power ($/MW)
}

df = pd.DataFrame(data)
display(df)


,Month,Water_Flow,Temperature,Sunshine,Rainfall,Hydro_Price,Solar_Price,Wind_Price
0,Jan,100,15,6,50,30,40,35
1,Feb,120,18,7,40,28,42,34
2,Mar,90,20,8,30,25,43,33
3,Apr,80,25,9,20,26,44,36
4,May,110,28,10,10,27,45,38
5,Jun,130,30,11,5,29,47,37
6,Jul,150,32,12,5,31,48,39
7,Aug,140,28,11,10,30,46,38
8,Sep,120,25,10,15,29,44,36
9,Oct,100,20,8,25,27,42,34


In [ ]:

# If rainfall and water flow are greater, we should take more power (MW) production from hydro assets and less from solar/wind. 
# However, if rainfall and water_flow are less, we should take more power (MW) production from wind and solar. Let's see how this can be done.


In [12]:

# Monthly Optimization
model_monthly = gp.Model("monthly_optimization")

# Decision variables for power generation from hydro, solar, and wind in each month
power_hydro = model_monthly.addVars(df['Month'], name="Hydro_Power", lb=0)
power_solar = model_monthly.addVars(df['Month'], name="Solar_Power", lb=0)
power_wind = model_monthly.addVars(df['Month'], name="Wind_Power", lb=0)

# Objective: Maximize total earnings over the year
model_monthly.setObjective(
    gp.quicksum(power_hydro[month] * df.loc[df['Month'] == month, 'Hydro_Price'].values[0] +
                power_solar[month] * df.loc[df['Month'] == month, 'Solar_Price'].values[0] +
                power_wind[month] * df.loc[df['Month'] == month, 'Wind_Price'].values[0]
                for month in df['Month']),
    sense=GRB.MAXIMIZE
)

# Constraints: Power generation based on environmental factors
for idx, month in df.iterrows():
    model_monthly.addConstr(power_hydro[month['Month']] <= 0.3 * month['Water_Flow'], f"Hydro_Constraint_{month['Month']}")
    model_monthly.addConstr(power_solar[month['Month']] <= 0.6 * month['Sunshine'], f"Solar_Constraint_{month['Month']}")
    model_monthly.addConstr(power_wind[month['Month']] <= 0.2 * (month['Temperature'] + month['Sunshine']), f"Wind_Constraint_{month['Month']}")

# Optimize the model
model_monthly.optimize()

# Print the optimal solution for each month
print("Optimal Monthly Earnings:")
for month in df['Month']:
    print(f"{month}: Hydro: {power_hydro[month].x:.2f} MW, Solar: {power_solar[month].x:.2f} MW, Wind: {power_wind[month].x:.2f} MW")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 36 rows, 36 columns and 36 nonzeros
Model fingerprint: 0xd5957413
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+01]
Presolve removed 36 rows and 36 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6610600e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.661060000e+04
Optimal Monthly Earnings:
Jan: Hydro: 30.00 MW, Solar: 3.60 MW, Wind: 4.20 MW
Feb: Hydro: 36.00 MW, Solar: 4.20 MW, Wind: 5.00 MW
Mar: Hydro: 27.00 MW, Solar: 4.80 MW, Wind: 5.60 MW
Apr:

In [13]:

# Quarterly Optimization
quarters = {
    'Q1': ['Jan', 'Feb', 'Mar'],
    'Q2': ['Apr', 'May', 'Jun'],
    'Q3': ['Jul', 'Aug', 'Sep'],
    'Q4': ['Oct', 'Nov', 'Dec']
}

model_quarterly = gp.Model("quarterly_optimization")

# Decision variables for power generation from hydro, solar, and wind in each quarter
power_hydro_q = model_quarterly.addVars(quarters.keys(), name="Hydro_Power", lb=0)
power_solar_q = model_quarterly.addVars(quarters.keys(), name="Solar_Power", lb=0)
power_wind_q = model_quarterly.addVars(quarters.keys(), name="Wind_Power", lb=0)

# Objective: Maximize total earnings over the year
model_quarterly.setObjective(
    gp.quicksum(power_hydro_q[q] * df.loc[df['Month'].isin(quarters[q]), 'Hydro_Price'].mean() +
                power_solar_q[q] * df.loc[df['Month'].isin(quarters[q]), 'Solar_Price'].mean() +
                power_wind_q[q] * df.loc[df['Month'].isin(quarters[q]), 'Wind_Price'].mean()
                for q in quarters.keys()),
    sense=GRB.MAXIMIZE
)

# Constraints: Power generation based on environmental factors for each quarter
for q, months in quarters.items():
    total_water_flow = df.loc[df['Month'].isin(months), 'Water_Flow'].sum()
    total_sunshine = df.loc[df['Month'].isin(months), 'Sunshine'].sum()
    total_temp_sunshine = df.loc[df['Month'].isin(months), ['Temperature', 'Sunshine']].sum().sum()
    model_quarterly.addConstr(power_hydro_q[q] <= 0.3 * total_water_flow, f"Hydro_Constraint_{q}")
    model_quarterly.addConstr(power_solar_q[q] <= 0.6 * total_sunshine, f"Solar_Constraint_{q}")
    model_quarterly.addConstr(power_wind_q[q] <= 0.2 * total_temp_sunshine, f"Wind_Constraint_{q}")

# Optimize the model
model_quarterly.optimize()

# Print the optimal solution for each quarter
print("Optimal Quarterly Earnings:")
for q in quarters.keys():
    print(f"{q}: Hydro: {power_hydro_q[q].x:.2f} MW, Solar: {power_solar_q[q].x:.2f} MW, Wind: {power_wind_q[q].x:.2f} MW")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 12 columns and 12 nonzeros
Model fingerprint: 0xadd608c0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 12 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6561533e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.656153333e+04
Optimal Quarterly Earnings:
Q1: Hydro: 93.00 MW, Solar: 12.60 MW, Wind: 14.80 MW
Q2: Hydro: 96.00 MW, Solar: 18.00 MW, Wind: 22.60 MW
Q3: Hydro: 123.00 MW, Solar: 19.80 MW, Wind: 23.60 M

In [14]:

# Yearly Optimization
model_yearly = gp.Model("yearly_optimization")

# Decision variables for power generation from hydro, solar, and wind for the year
power_hydro_y = model_yearly.addVar(name="Hydro_Power", lb=0)
power_solar_y = model_yearly.addVar(name="Solar_Power", lb=0)
power_wind_y = model_yearly.addVar(name="Wind_Power", lb=0)

# Objective: Maximize total earnings for the year
model_yearly.setObjective(
    power_hydro_y * df['Hydro_Price'].mean() +
    power_solar_y * df['Solar_Price'].mean() +
    power_wind_y * df['Wind_Price'].mean(),
    sense=GRB.MAXIMIZE
)

# Constraints: Power generation based on environmental factors for the year
total_water_flow = df['Water_Flow'].sum()
total_sunshine = df['Sunshine'].sum()
total_temp_sunshine = df[['Temperature', 'Sunshine']].sum().sum()
model_yearly.addConstr(power_hydro_y <= 0.3 * total_water_flow, "Hydro_Constraint")
model_yearly.addConstr(power_solar_y <= 0.6 * total_sunshine, "Solar_Constraint")
model_yearly.addConstr(power_wind_y <= 0.2 * total_temp_sunshine, "Wind_Constraint")

# Optimize the model
model_yearly.optimize()

# Print the optimal solution for the year
print("Optimal Yearly Earnings:")
print(f"Hydro: {power_hydro_y.x:.2f} MW, Solar: {power_solar_y.x:.2f} MW, Wind: {power_wind_y.x:.2f} MW")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xa47c9ad5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 4e+02]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6430917e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.643091667e+04
Optimal Yearly Earnings:
Hydro: 393.00 MW, Solar: 63.00 MW, Wind: 76.00 MW


In [ ]:

# END!!! 
